In [ ]:
################################################################################
# 6 Multiple Resource Theory
# ----------------------------
# where's reference from?
# Multiple Resource Theory is to predict the relative differences in task
# interference between different task configurations. It proposes four categorical
# and dichotomous dimensions, each dimension has two discrete 'levels':
#       processing stages -- perception, cognition VS responding
#       perceptual modalities -- auditory VS visual
#       visual channels -- focal VS ambient
#       processing codes -- spatial VS verbal
#
# GUI Usage:
#
# Model limitations:
# 1) assuming all tasks are alike
################################################################################

import tkinter as tk
from tkinter import ttk
import numpy as np
from itertools import product, combinations     # don't know what's this for?
import matplotlib
matplotlib.use("TkAgg")                         # for 3d cube visualization
import matplotlib.backends.backend_tkagg as tkagg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

class MRT(tk.Frame):
    tasks = ['driving', 'reading', 'listening']
    taskAmount = 3

    def __init__(self, parent=None):
        tk.Frame.__init__(self, parent)
        self.pack()
        self.makeWidgets()
        self.defineTasks()
        self.flagTaskSelected = [0 for x in range(self.taskAmount)] # flags of selected tasks
        for x in range(self.taskAmount):
            self.flagTaskSelected[x] = False
        self.counterTaskSelected = 0                                # to limit maximum two tasks are selected
        self.analysisTasks()
        self.visualizeCube()
        self.computeConflictMatrix()

    # GUI layout
    def makeWidgets(self):
        self.frameTaskSelection = tk.Canvas(self)
        self.frameCube = tk.Frame(self)
        self.frameTaskAnalysisShell = tk.Frame(self)
        self.frameConflictMatrix = tk.Canvas(self, relief=tk.GROOVE, bd=2, bg='white')
        self.frameInterferenceValue = tk.Frame(self)
        self.frameTaskSelection.grid(row=0, column=0, padx=10, pady=10)
        self.frameCube.grid(row=0, column=1, padx=10, pady=10)
        ttk.Separator(self, orient=tk.HORIZONTAL).grid(row=1, column=0, columnspan=3, sticky=tk.EW, padx=10)
        self.frameTaskAnalysisShell.grid(row=2, column=0, padx=10, pady=10)
        self.frameConflictMatrix.grid(row=2, column=1, padx=10, pady=10)
        self.frameInterferenceValue.grid(row=2, column=2, padx=10, pady=10)

    # define arrows indicating operation flow
    def defineSigns(self):
        # self.sign_taskSelection_cube
        # self.sign_taskAnalysisShell_conflictMatrix
        # self.sign_conflictMatrix_InterferenceValue
        print ("define arrows")

    # define tasks
    def defineTasks(self):
        taskBrick_positionX = 5
        taskBrick_positionY = 5
        taskBrick_width = 200
        taskBrick_height = 50
        fontSize = 16
        self.taskBrick = [0 for x in range(self.taskAmount)]
        self.taskBrickText = [0 for x in range(self.taskAmount)]
        for x in range(3):
            self.taskBrick[x] = self.frameTaskSelection.create_rectangle(taskBrick_positionX, taskBrick_positionY+x*taskBrick_height, taskBrick_positionX+taskBrick_width, taskBrick_positionY+taskBrick_height+x*taskBrick_height, fill='#63B2A9', tags=self.tasks[x])
            self.taskBrickText[x] = self.frameTaskSelection.create_text(taskBrick_positionX + 0.5*taskBrick_width, taskBrick_positionY+0.5*taskBrick_height+x*taskBrick_height, text=self.tasks[x], tags=self.tasks[x], font=("Arial", fontSize))
            self.frameTaskSelection.tag_bind(self.tasks[x], '<ButtonPress-1>', self.selectTasks)

    # detect task selection
    def selectTasks(self, event):
        eventObj = self.frameTaskSelection.find_closest(event.x, event.y)
        eventObjTag = self.frameTaskSelection.itemcget(eventObj, 'tags')
        for i in range(self.taskAmount):
            if eventObjTag == self.tasks[i] + ' current':
                if self.flagTaskSelected[i] == True:
                    self.flagTaskSelected[i] = False
                    self.counterTaskSelected -= 1
                    self.frameTaskSelection.itemconfigure(self.taskBrick[i], fill='#63B2A9')
                    self.frameTaskSelection.move(self.tasks[i], -30, 0)
                elif self.flagTaskSelected[i] == False and self.counterTaskSelected < 2:
                    self.flagTaskSelected[i] = True
                    self.counterTaskSelected += 1
                    self.frameTaskSelection.itemconfigure(self.taskBrick[i], fill='#C0FFF8')
                    self.frameTaskSelection.move(self.tasks[i], 30, 0)
        # output self.flagTaskSelected[] indicating selected two tasks

# task analysis shell
    def analysisTasks(self):
        self.tableTaskAnalysisShell = ttk.Treeview(self.frameTaskAnalysisShell, columns=('stages', 'modalities', 'codes'), selectmode = "browse")
        self.tableTaskAnalysisShell.grid(row=0, column=0)
        self.tableTaskAnalysisShell.column('#0', width=80)
        self.tableTaskAnalysisShell.column('stages', width=200, stretch=True)
        self.tableTaskAnalysisShell.column('modalities', width=120, stretch=True)
        self.tableTaskAnalysisShell.column('codes', width=120, stretch=True)
        self.tableTaskAnalysisShell.heading('stages', text='Stages')
        self.tableTaskAnalysisShell.heading('modalities', text='Modalities')
        self.tableTaskAnalysisShell.heading('codes', text='Codes')
        self.tableTaskAnalysisShell.insert('', 0, 'driving', text='Driving', tags=('tagDriving'), values='perception&cognition+responding visual spatial')
        self.tableTaskAnalysisShell.insert('', 1, 'modalities', text='Reading', tags=('tagReading'), values='perception&cognition visual verbal')
        self.tableTaskAnalysisShell.insert('', 2, 'listening', text='Listening', tags=('tagListening'), values='perception&cognition auditory verbal')
        self.tableTaskAnalysisShell.tag_configure('tagListening', background='#C0FFF8')

# 3D Cube visualization
    def visualizeCube(self):
        # embed matplotlib 3D plotting in tkinter GUI
        fig = plt.Figure(figsize=(3, 2), dpi=100)
        ax = fig.add_subplot(111, projection='3d')
        ax.set_aspect("equal")
        r = [-1, 1]
        for s, e in combinations(np.array(list(product(r, r, r))), 2):
            if np.sum(np.abs(s-e)) == r[1]-r[0]:
                ax.plot3D(*zip(s, e), color='#63B2A9')
        canvas = tkagg.FigureCanvasTkAgg(fig, master=self.frameCube)
        canvas.show()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)


# conflict matrix
    def computeConflictMatrix(self):
        # make a table of conflict matrix, with real-time slot marking
        cellWidth = 40
        cellHeight = 25
        self.cell = [[0 for y in range(9)] for x in range(10)]
        for x in range(10):
            for y in range(9):
                self.cell[x][y] = self.frameConflictMatrix.create_rectangle((x-1)*cellWidth, (y-1)*cellHeight, x*cellWidth, y*cellHeight, outline='')
        self.textPerceptual = self.frameConflictMatrix.create_text(3*cellWidth, 0.5*cellHeight, text='Perceptual')
        self.textCognitive = self.frameConflictMatrix.create_text(6*cellWidth, 0.5*cellHeight, text='Cognitive')
        self.textResponse = self.frameConflictMatrix.create_text(8*cellWidth, 0.5*cellHeight, text='Response')
        levelsOfDimensions = ['VS', 'VV', 'AS', 'AV', 'CS', 'CV', 'RS', 'RV']
        self.textLevelsOfDimensions = [0 for m in range(2)]
        for m in range(2):
            self.textLevelsOfDimensions[m] = [0 for n in range(8)]
            for n in range(8):
                if m == 0:
                    self.textLevelsOfDimensions[m][n] = self.frameConflictMatrix.create_text(0.5*cellWidth, (2.5+n)*cellHeight, text=levelsOfDimensions[n])
                elif m == 1:
                    self.textLevelsOfDimensions[m][n] = self.frameConflictMatrix.create_text((1.5+n)*cellWidth, 1.5*cellHeight, text=levelsOfDimensions[n])
        # valuesOfConflicts = [[0 for q in range(8)] for p in range(8)]
        valuesOfConflicts = ['0.8', '0.6', '0.6', '0.4', '0.7', '0.5', '0.4', '0.2',
                            '', '0.8', '0.4', '0.6', '0.5', '0.7', '0.2', '0.4',
                            ' ', ' ', '0.8', '0.4', '0.7', '0.5', '0.4', '0.2',
                            ' ', ' ', '0', '0.8', '0.5', '0.7', '0.2', '0.4',
                            ' ', ' ', '0', ' ', '0.8', '0.6', '0.6', '0.4',
                            ' ', ' ', ' ', ' ', ' ', '0.8', '0.4', '0.6',
                            ' ', ' ', ' ', ' ', ' ', ' ', '0.8', '0.6',
                            ' ', ' ', ' ', ' ', ' ', ' ', '0.6', '1.0']
        self.textValuesOfConflicts = [[0 for q in range(8)] for p in range(8)]
        for p in range(8):
            for q in range(8):
                self.textValuesOfConflicts[p][q] = self.frameConflictMatrix.create_text((1.5+q)*cellWidth, (2.5+p)*cellHeight, text=valuesOfConflicts[p*8+q])
        self.frameConflictMatrix.create_line(cellWidth, cellHeight, 9*cellWidth, cellHeight, width=2)
        self.frameConflictMatrix.create_line(10, 6*cellHeight, 9*cellWidth, 6*cellHeight, dash=(4, 4))
        self.frameConflictMatrix.create_line(10, 8*cellHeight, 9*cellWidth, 8*cellHeight)
        self.frameConflictMatrix.create_line(5*cellWidth, 10, 5*cellWidth, 10*cellHeight, dash=(4, 4))
        self.frameConflictMatrix.create_line(7*cellWidth, 10, 7*cellWidth, 10*cellHeight)
        self.frameConflictMatrix.itemconfigure(self.cell[7][5], fill='#C0FFF8')

# computational formula
    def computeInterferenceValue(self):
        print ("computational formula")
        # demand component = summing the average demand of both tasks
        # conflict component =


if __name__ == "__main__":
    root = tk.Tk()
    root.title("Multiple Resource Theory")
    root.geometry("1000x600")
    root.resizable(width=False, height=False)
    MRT(root).mainloop()


D:\Anaconda3\lib\site-packages\mpl_toolkits\mplot3d\axes3d.py:1025: UserWarning: Axes3D.figure.canvas is 'None', mouse rotation disabled.  Set canvas then call Axes3D.mouse_init().
  warnings.warn('Axes3D.figure.canvas is \'None\', mouse rotation disabled.  Set canvas then call Axes3D.mouse_init().')
